In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset, Subset
import torch.nn.functional as F
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
import cufflinks as cf
from sklearn.preprocessing import LabelEncoder
cf.go_offline(connected = True)



In [2]:
df_total = pd.read_excel('Final.xlsx',index_col = 0)

In [3]:
df_list_normal = pd.read_excel('MuSinSa_photo/Total_page_info.xlsx',index_col= 0)

lst_total = df_list_normal['index'].values

len(lst_total), lst_total

df_musina = pd.read_excel('First_to_Second_page.xlsx',index_col = 0).reset_index()

df_musina['index'] = [i for i in range(df_musina.shape[0])]

df_musina_2 = df_musina.iloc[df_list_normal['index'].values,:].copy()



In [4]:
df_total['style'].unique()

array(['아메리칸 캐주얼', '캐주얼', '댄디', '포멀', '걸리시', '골프', '홈웨어', '레트로', '로맨틱',
       '스포츠', '스트릿', '시크'], dtype=object)

In [5]:
df_total.head(2)

,uniq_index,class_id,prob,style,style_part,comment,tag,coordinate,domain_url
0,0,"[8, 4, 2]","[0.8417168259620667, 0.8145477771759033, 0.741...",아메리칸 캐주얼,아메카지 멋 내기,브라운 컬러가 돋보이는 카디건과 코튼 팬츠를 코디하고 숏 패딩과 머플러를 더해 완성...,"['겨울', '아메리칸 캐주얼', '구두', '로스트가든', '메신저/크로스 백',...","[[372, 136, 600, 307], [58, 23, 536, 427], [54...",https://store.musinsa.com/app/codimap/views/20899
1,1,"[4, 8]","[0.9969467520713806, 0.7027121186256409]",아메리칸 캐주얼,아메카지 멋 내기,브라운 컬러가 돋보이는 카디건과 코튼 팬츠를 코디하고 숏 패딩과 머플러를 더해 완성...,"['겨울', '아메리칸 캐주얼', '구두', '로스트가든', '메신저/크로스 백',...","[[81, 74, 601, 422], [437, 122, 601, 362]]",https://store.musinsa.com/app/codimap/views/20899


In [6]:
items=['아메리칸 캐주얼', '캐주얼', '댄디', '포멀', '걸리시', '골프', '홈웨어', '레트로', '로맨틱',
       '스포츠', '스트릿', '시크']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print(labels)
len(labels)

[ 8  9  2 10  0  1 11  3  4  6  5  7]


12

In [7]:
df_total_2 = df_total.copy()

In [8]:
encoder.transform(['아메리칸 캐주얼'])[0]

8

In [9]:
def manipulate(row):
    row['style'] = encoder.transform([row['style']])[0]
    return row

In [10]:
df_total_3 = df_total_2.apply(manipulate,axis=1).copy()

In [11]:
df_total_3.head(1)

,uniq_index,class_id,prob,style,style_part,comment,tag,coordinate,domain_url
0,0,"[8, 4, 2]","[0.8417168259620667, 0.8145477771759033, 0.741...",8,아메카지 멋 내기,브라운 컬러가 돋보이는 카디건과 코튼 팬츠를 코디하고 숏 패딩과 머플러를 더해 완성...,"['겨울', '아메리칸 캐주얼', '구두', '로스트가든', '메신저/크로스 백',...","[[372, 136, 600, 307], [58, 23, 536, 427], [54...",https://store.musinsa.com/app/codimap/views/20899


In [12]:
fe = pd.Series(df_total_3['class_id'].values)[0]

In [13]:
[int(W) for W in fe.replace('[','').replace(']','').split(',')]

[8, 4, 2]

In [14]:
import tqdm

A = pd.DataFrame(np.zeros((6973, 14)))
for idx,(i,j,y) in enumerate(tqdm.tqdm(df_total_3[['class_id','prob','style']].values)):
    

    
    
    if i == 0:
        A.loc[idx,14] = y
    
    else:
        i  = [float(W) for W in i.replace('[','').replace(']','').split(',')]
        j  = [float(W) for W in j.replace('[','').replace(']','').split(',')]    
        if len(i) == 1:
            A.loc[idx,i[0]] = j[0]
            A.loc[idx,14] = y
        else:
            for idx_1 in range(len(i)):
                A.loc[idx,i[idx_1]] = j[idx_1]
            A.loc[idx,14] = y
    

        
        

100%|████████████████████████████████████████████████████████████████████████████| 6973/6973 [00:02<00:00, 3351.67it/s]


In [18]:
dataset = TensorDataset(torch.from_numpy(A.iloc[:,:13].values + 0.01),torch.from_numpy(A.iloc[:,14].values))

In [19]:
from sklearn.model_selection import train_test_split

VAR = 0.2
train_indices, val_indices = train_test_split(range(A.iloc[:,:13].shape[0]),
                                             test_size = VAR)

train_dataset = Subset(dataset, train_indices)
validation_dataset = Subset(dataset, val_indices)

minibatch_size = 128


train_batches = DataLoader(train_dataset, batch_size = minibatch_size, shuffle = True)
val_batches = DataLoader(validation_dataset, batch_size = minibatch_size, shuffle = True)

In [20]:
class Net(nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()
        
        self.linear_layers = nn.Sequential(nn.Linear(input_dim, 200),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(200,100),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(100,50),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(50,20),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(20,5),
                                          nn.LeakyReLU(0.1),
                                          nn.Linear(5, output_dim), 
                                           # 최종 결과는 (20, 3) 이 되므로, dim=-1 로 label 확률값이 들어 있는 마지막 차원을 지정해줘야 함
                                          nn.LogSoftmax(dim = -1)
                                          )
        
    def forward(self,x):
        y = self.linear_layers(x)
        
        return y

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)
print (device)

cuda


In [50]:
input_dim = torch.from_numpy(A.iloc[:,:13].values).size(-1)
output_dim = 12
# 여기서는 12개의 label 이 있음!!

print(input_dim, output_dim )

model = Net(input_dim, output_dim).to(device)

loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

13 12


In [51]:
nb_epochs = 10000

X_train_tensor = torch.from_numpy(A.iloc[:,:13].values +0.1).type(torch.float32)
y_train_tensor = torch.from_numpy(A.iloc[:,14].values).type(torch.LongTensor)

model.train()
for index in range(nb_epochs):
    indices = torch.randperm(X_train_tensor.size(0))

    x_batch_list = torch.index_select(X_train_tensor, 0, index=indices)
    y_batch_list = torch.index_select(y_train_tensor, 0, index=indices)
    x_batch_list = x_batch_list.split(minibatch_size, 0)
    y_batch_list = y_batch_list.split(minibatch_size, 0)

    epoch_loss = list()        
    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        
        x_minibatch = x_minibatch.to(device)
        y_minibatch = y_minibatch.to(device)
        
        y_minibatch_pred = model(x_minibatch)
        loss = loss_func(y_minibatch_pred, y_minibatch)
        epoch_loss.append(loss)        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (index % 40) == 0:
        print (index, sum(epoch_loss) / len(epoch_loss))        


0 tensor(2.5213, device='cuda:0', grad_fn=<DivBackward0>)
40 tensor(2.3594, device='cuda:0', grad_fn=<DivBackward0>)
80 tensor(2.3383, device='cuda:0', grad_fn=<DivBackward0>)
120 tensor(2.3237, device='cuda:0', grad_fn=<DivBackward0>)
160 tensor(2.3165, device='cuda:0', grad_fn=<DivBackward0>)
200 tensor(2.3055, device='cuda:0', grad_fn=<DivBackward0>)
240 tensor(2.3005, device='cuda:0', grad_fn=<DivBackward0>)
280 tensor(2.2937, device='cuda:0', grad_fn=<DivBackward0>)
320 tensor(2.2898, device='cuda:0', grad_fn=<DivBackward0>)
360 tensor(2.2866, device='cuda:0', grad_fn=<DivBackward0>)
400 tensor(2.2782, device='cuda:0', grad_fn=<DivBackward0>)
440 tensor(2.2699, device='cuda:0', grad_fn=<DivBackward0>)
480 tensor(2.2700, device='cuda:0', grad_fn=<DivBackward0>)
520 tensor(2.2650, device='cuda:0', grad_fn=<DivBackward0>)
560 tensor(2.2574, device='cuda:0', grad_fn=<DivBackward0>)
600 tensor(2.2533, device='cuda:0', grad_fn=<DivBackward0>)
640 tensor(2.2503, device='cuda:0', grad_fn=

KeyboardInterrupt: 

In [30]:
X_train_tensor

tensor([[0.1000, 0.1000, 0.8415,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.8801,  ..., 0.1000, 0.1000, 0.1000],
        ...,
        [0.1000, 0.1000, 0.9631,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000]])

In [31]:
# 'class_ids': array([8, 1]),
#   'scores': array([0.99361986, 0.8832041 ]

In [52]:
model.to('cpu')

Net(
  (linear_layers): Sequential(
    (0): Linear(in_features=13, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.1)
    (6): Linear(in_features=50, out_features=20, bias=True)
    (7): LeakyReLU(negative_slope=0.1)
    (8): Linear(in_features=20, out_features=5, bias=True)
    (9): LeakyReLU(negative_slope=0.1)
    (10): Linear(in_features=5, out_features=12, bias=True)
    (11): LogSoftmax(dim=-1)
  )
)

In [62]:
torch.argmax(model(torch.Tensor([0,0.8832041 , 0, 0, 0, 0, 0, 0, 0.99361986, 0, 0, 0, 0])))

tensor(4)

In [65]:
torch.save(model.state_dict(), 'model_1.pt')